In [215]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

In [175]:
name_file_data_report = 'Общая таблица слушателей ЦОПП.xlsx' 

In [176]:
df_dpo = pd.read_excel(name_file_data_report,sheet_name='ДПО')

In [177]:
#Создаем файл 

In [178]:
# Раздел 1.3
# Количество программ по каждому виду обучения
# группируем. Так как нам нужны текстовые данные то применяем создаем строку с помощью join
quantity_program_on_type_provisional =df_dpo.groupby('Наименование_дополнительной_профессиональной_программы').agg({'Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка':lambda x:','.join(x)})

# Применяем к полученной серии функцию разделения по запятой. Предполо
quantity_program_on_type_provisional['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка']=quantity_program_on_type_provisional['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'].apply(lambda x: x.split(',')[0])

df_1_3=quantity_program_on_type_provisional['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'].value_counts().to_frame()
# Раздел 1.3

In [179]:
# Создаем пустую колонку 4
temp_df = pd.DataFrame(columns=['4'])
df_1_3 = pd.concat([df_1_3,temp_df],axis=1)
# переименываем первую колонку
df_1_3.rename(columns={'Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка':'Число реализованных программ'},inplace=True)


In [180]:
group_quantity_student_program = df_dpo.groupby(['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка']).size()
group_quantity_student_program

Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка
Повышение квалификации              27
Профессиональная переподготовка      9
dtype: int64

In [181]:
# Добавляем колонку 5- всего слушателей
df_1_3['Всего слушателей'] = group_quantity_student_program
df_1_3

,Число реализованных программ,4,Всего слушателей
Повышение квалификации,2,NaN,27
Профессиональная переподготовка,1,NaN,9


In [182]:
# Подсчет сетевой формы
df_dpo_network = df_dpo[df_dpo['Использование_сетевой_формы_обучения'] == 'Сетевая форма']

In [183]:
group_quantity_network_program_provisional=df_dpo_network.groupby('Наименование_дополнительной_профессиональной_программы').agg({'Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка':lambda x:','.join(x)})
# Применяем к полученной серии функцию разделения по запятой. Предполо
group_quantity_network_program_provisional['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка']=group_quantity_network_program_provisional['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'].apply(lambda x: x.split(',')[0])
df_1_3['Число программ сетевая форма'] = group_quantity_network_program_provisional['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'].value_counts().to_frame()

In [184]:
df_1_3

,Число реализованных программ,4,Всего слушателей,Число программ сетевая форма
Повышение квалификации,2,NaN,27,1
Профессиональная переподготовка,1,NaN,9,1


In [185]:
# Считаем число слушателей на сетевых программах
df_1_3['Численость слушателей сетевый программ']=df_dpo_network.groupby('Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка').agg({'ФИО_именительный':'count'})

In [186]:
df_1_3

,Число реализованных программ,4,Всего слушателей,Число программ сетевая форма,Численость слушателей сетевый программ
Повышение квалификации,2,NaN,27,1,3
Профессиональная переподготовка,1,NaN,9,1,9


In [187]:
# Добавляем 4 пустых столбца
temp_df = pd.DataFrame(columns=['8','9','10','11','12'])
df_1_3 = pd.concat([df_1_3,temp_df],axis=1)


In [188]:
df_1_3

,Число реализованных программ,4,Всего слушателей,Число программ сетевая форма,Численость слушателей сетевый программ,8,9,10,11,12
Повышение квалификации,2,NaN,27,1,3,NaN,NaN,NaN,NaN,NaN
Профессиональная переподготовка,1,NaN,9,1,9,NaN,NaN,NaN,NaN,NaN


In [189]:
# Считаем дистанционные технологии
df_dpo_distant = df_dpo[df_dpo['Использование_ЭО_или_ДОТ'] != 'Без применения ЭО или ДОТ']
group_quantity_distant = df_dpo_distant.groupby('Наименование_дополнительной_профессиональной_программы').agg({'Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка':lambda x:','.join(x)})

# Применяем к полученной серии функцию разделения по запятой. 
group_quantity_distant['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка']=group_quantity_distant['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'].apply(lambda x: x.split(',')[0])
df_1_3['Число программ ЭО и ДОТ'] = group_quantity_distant['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'].value_counts().to_frame()


In [190]:
df_1_3

,Число реализованных программ,4,Всего слушателей,Число программ сетевая форма,Численость слушателей сетевый программ,8,9,10,11,12,Число программ ЭО и ДОТ
Повышение квалификации,2,NaN,27,1,3,NaN,NaN,NaN,NaN,NaN,1
Профессиональная переподготовка,1,NaN,9,1,9,NaN,NaN,NaN,NaN,NaN,1


In [191]:
#Считаем количество слушателей
df_1_3['Численность слушателей ЭО И ДОТ'] = df_dpo_distant.groupby('Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка').agg({'ФИО_именительный':'count'})

In [192]:
df_1_3

,Число реализованных программ,4,Всего слушателей,Число программ сетевая форма,Численость слушателей сетевый программ,8,9,10,11,12,Число программ ЭО и ДОТ,Численность слушателей ЭО И ДОТ
Повышение квалификации,2,NaN,27,1,3,NaN,NaN,NaN,NaN,NaN,1,3
Профессиональная переподготовка,1,NaN,9,1,9,NaN,NaN,NaN,NaN,NaN,1,9


In [193]:
# Считаем только тех кто учился на программах исключительно ЭО и ДОТ
df_dpo_distant_only_dot = df_dpo_distant[df_dpo_distant['Использование_ЭО_или_ДОТ'] == 'Исключительно с ЭО ли ДОТ' ]
df_1_3['Численность слушателей только ЭО и ДОТ'] = df_dpo_distant_only_dot.groupby('Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка').agg({'ФИО_именительный':'count'})

In [201]:
finish_df_1_3 = df_1_3.reset_index()

In [202]:
finish_df_1_3

,index,Число реализованных программ,4,Всего слушателей,Число программ сетевая форма,Численость слушателей сетевый программ,8,9,10,11,12,Число программ ЭО и ДОТ,Численность слушателей ЭО И ДОТ,Численность слушателей только ЭО и ДОТ
0,Повышение квалификации,2,NaN,27,1,3,NaN,NaN,NaN,NaN,NaN,1,3,NaN
1,Профессиональная переподготовка,1,NaN,9,1,9,NaN,NaN,NaN,NaN,NaN,1,9,NaN


In [203]:
finish_df_1_3.columns= ['Наименование образовательных программ','Число реализованных образовательных программ всего,единиц','из них прошли профессионально-общественную аккредитацию',
                'Всего слушателей, обученных по программам, человек','число программ(из графы 3) реализованных с использованием сетевой формы',
                'численность слушателей обученных(из графы 5) по программам с использованием сетевой формы-всего',
                 'в том числе(из графы 7) с использованием ресурсов иностранных организаций','общее число заключенных договоров с организациями на реализацию образовательных программ с использованием сетевой формы',
                 'из них число заключенных договоров(из графы 9) с использованием ресурсов иностранных организаций','число организаций с которыми заключены договоры  на реализацию образовательных программ с использованием сетевой формы',
                'в том числе(из графы 11) число иностранных организаций','число программ(из графы 3),реализованных с применением ЭО или ДОТ','Численность слушателей обученных(из графы 5) по программам с применением ЭО и ДОТ',
                 'в том числе(из графы 14) с применением исключительно ЭО и ДОТ']

In [204]:
finish_df_1_3

,Наименование образовательных программ,"Число реализованных образовательных программ всего,единиц",из них прошли профессионально-общественную аккредитацию,"Всего слушателей, обученных по программам, человек",число программ(из графы 3) реализованных с использованием сетевой формы,численность слушателей обученных(из графы 5) по программам с использованием сетевой формы-всего,в том числе(из графы 7) с использованием ресурсов иностранных организаций,общее число заключенных договоров с организациями на реализацию образовательных программ с использованием сетевой формы,из них число заключенных договоров(из графы 9) с использованием ресурсов иностранных организаций,число организаций с которыми заключены договоры на реализацию образовательных программ с использованием сетевой формы,в том числе(из графы 11) число иностранных организаций,"число программ(из графы 3),реализованных с применением ЭО или ДОТ",Численность слушателей обученных(из графы 5) по программам с применением ЭО и ДОТ,в том числе(из графы 14) с применением исключительно ЭО и ДОТ
0,Повышение квалификации,2,NaN,27,1,3,NaN,NaN,NaN,NaN,NaN,1,3,NaN
1,Профессиональная переподготовка,1,NaN,9,1,9,NaN,NaN,NaN,NaN,NaN,1,9,NaN


In [225]:
# Создаем файл excel
wb = openpyxl.Workbook()
# Создаем лист раздела 1.3
wb.create_sheet(title='Раздел 1.3',index=0)



<Worksheet "Раздел 1.3">

In [226]:
for r in dataframe_to_rows(finish_df_1_3, index=False, header=True):
    wb['Раздел 1.3'].append(r)

In [227]:
wb.save('Раздел 1.3.xlsx')